# Llama-2-7b Sentiment Analysis

# Installation

In [1]:
pip install -q -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.31 trl==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 123.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


In [2]:
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from google.colab import drive
import os
drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Y4S1/CZ4042 NN Grp Proj/llama-2")
print("Changed to:", os.getcwd())  # Print the new current directory

Mounted at /content/drive
Changed to: /content/drive/My Drive/Y4S1/CZ4042 NN Grp Proj/llama-2


# Data Preparation

In [8]:
from datasets import load_dataset

# Load the dataset
data = load_dataset("csv", data_files="IMDB Dataset.csv")
data_train = data['train'].select([i for i in range(400)])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
filename = "IMDB Dataset.csv"

df = pd.read_csv(filename)
df = df.head(500)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

#reset index
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

#X_test and y_test df
X_test = test_df["review"]
y_test = test_df["sentiment"]

In [6]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the movie review enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label 'positive' or 'negative' only.

            Review: [{data_point["review"]}]
            Sentiment = {data_point["sentiment"]}

            """.strip()


def generate_test_prompt(review):
    return f"""
            Analyze the sentiment of the movie review enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label 'positive' or 'negative' only.

            Review: [{review}]
            Sentiment =
             """.strip()


def evaluate(y_true, y_pred):
    labels = ['positive', 'negative']
    mapping = {'positive': 1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

# Loading the pre-trained model

In [13]:
# load llama-2 language model from hugging face hub
model_name = "abhishek/llama-2-7b-hf-small-shards"
#model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "Xilabs/Llama-2-7b-Sharded"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

device = "cuda:0"


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    offload_folder="offload",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Prediction without fine-tuning

In [26]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test[i]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.5,
                       )
        result = pipe(prompt)
        print("result", result)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        else:
            y_pred.append("none")
    return y_pred

In [23]:
from tqdm import tqdm
from transformers import pipeline

generation_config = model.generation_config
generation_config.max_new_tokens = 3
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

def predict(X_test, model, tokenizer):
  y_pred = []
  device = "cuda:0"

  for i in tqdm(range(len(X_test))):
     # print("\nPrompt:", X_test[i])
      prompt = generate_test_prompt(X_test[i])

      encoding = tokenizer(prompt, return_tensors="pt").to(device)
      with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
      result = tokenizer.decode(outputs[0], skip_special_tokens=True)
   #   print("Result:", result)
      answer = result.split("=")[-1]
    #  print("Answer:", answer)
      if "positive" in answer:
          y_pred.append("positive")
      elif "negative" in answer:
          y_pred.append("negative")
      else:
          y_pred.append("none")
  return y_pred

In [ ]:
y_pred = predict(X_test, model, tokenizer)

In [16]:
print("Results without fine-tuning")
evaluate(y_test, y_pred)

Results without fine-tuning
Accuracy: 0.630
Accuracy for label 0: 0.150
Accuracy for label 1: 0.950

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.15      0.24        40
           1       0.63      0.95      0.75        60

    accuracy                           0.63       100
   macro avg       0.65      0.55      0.50       100
weighted avg       0.64      0.63      0.55       100


Confusion Matrix:
[[ 6 34]
 [ 3 57]]


## Fine-tuning

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)


trainer = SFTTrainer(
    model=model,
    train_dataset=data_train,
    peft_config=peft_config,
    dataset_text_field="review",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

In [ ]:
#predict using trained model
y_finetuned_pred = predict(X_test, model, tokenizer)

In [21]:
print("Results with fine-tuning")
evaluate(y_test, y_finetuned_pred)

Results with fine-tuning
Accuracy: 0.840
Accuracy for label 0: 0.675
Accuracy for label 1: 0.950

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.68      0.77        40
           1       0.81      0.95      0.88        60

    accuracy                           0.84       100
   macro avg       0.86      0.81      0.82       100
weighted avg       0.85      0.84      0.83       100


Confusion Matrix:
[[27 13]
 [ 3 57]]
